In [ ]:
import os
import sys
sys.path.insert(1, '/mnt/d/PowerTAC/Python/python_utils/helper') 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from statistics import mean
from matplotlib import pyplot as plt
from plotly.subplots import make_subplots
from read_mongo_collection import HelperToReadMongo

sns.set_theme(style="darkgrid")

# See complete data
pd.set_option('max_columns', None)
pd.set_option("max_rows", None)
np.set_printoptions(threshold=np.inf)

In [ ]:
helper_mongo = HelperToReadMongo()

In [ ]:
rename_col = {"MCP": "Price", "LimitPrice": "Price" , "Net_Cleared_Quantity": "Quantity", \
              "Broker's_Cleared_Quantity": "Quantity", "Broker's_Bidded_Quantity": "Quantity", \
              "Market_Position": "Quantity"}

In [ ]:
color_book = {'VV21': 'darkorchid', 'VidyutVanika': 'slateblue', 'VV20': 'slateblue', 'VidyutVanika18': 'yellowgreen', 'VV18': 'yellowgreen', \
              'Sample': 'orchid', 'AgentUDE': 'slategray', 'SPOT': 'tomato', 'TUC_TAC': 'gold', 'MCP': 'red'}

def get_colors(list_of_brokers):
    return [color_book.get(broker) for broker in list_of_brokers]

# Data Processing

In [ ]:
# Specify names of all the databases here
list_of_databases = ['PowerTAC2021_Trials1_3_reduced_croc_high']

# Specify names of all the brokers corresponding to databases here
list_of_brokers = ['VV21']

# Specify the storage path along with directory
storage_path = '/mnt/d/PowerTAC/PowerTAC2021/experiments/capacity_transaction_check/'
os.makedirs(storage_path, exist_ok=True) 

# Specify list of the collections to be analysed here
# list_of_collections = helper_mongo.connect_to_mongo('PowerTAC2020_Wholesale_Test_VV21').list_collection_names()
list_of_collections = ['Capacity_Transaction_Net_Demand_Info', 'CapacityTransaction_Info']

# Keep all the collections at one place
dict_of_dataframes = dict()

In [ ]:
# View the Collections in all the databases

def view_collections(list_of_databases):
    
    for database in list_of_databases:
        
        db = helper_mongo.connect_to_mongo(database, server_ip='10.2.16.159', ssh_username='powertac', \
                                           ssh_password='sanjay9397', remote=False)
        print(database + "\n" + "-"*35)
        print(db.list_collection_names())
        print("\n")

# Function Calling
view_collections(list_of_databases)

In [ ]:
for dataset, broker in zip(list_of_databases, list_of_brokers):
    
    inner_dict = dict()
        
    for collection in list_of_collections:
        
        db = helper_mongo.query_to_mongo(dataset, collection, server_ip='10.2.16.159', ssh_username='powertac', \
                                         ssh_password='sanjay9397', remote=False)
        db.rename(columns = rename_col, inplace = True)
        
        if collection not in ['Capacity_Transaction_Net_Demand_Info', 'Cash_and_Market_Position_Info', 'CapacityTransaction_Info']:
            db = db[abs(db['Quantity']) > 0.05] 
            # if (collection != 'ClearedTrade_Info'):
            #    # consider only bids
            #    db = db[db['Price'] < 0.0]
            db['Proximity'] = db['Execution_Timeslot'] - db['Bidding_Timeslot']
            
        inner_dict.update({collection: db})
                
    dict_of_dataframes.update({broker: inner_dict})

# Analysis and Plotting

In [ ]:
###############################################
############ Plot using Plotly ################
###############################################
    
def capacity_transaction_plotly(broker, game):    
        
    fig = go.Figure()
                            
    database_nb = dict_of_dataframes.get(broker).get('Capacity_Transaction_Net_Demand_Info').copy(deep=True)
    database_ct = dict_of_dataframes.get(broker).get('CapacityTransaction_Info').copy(deep=True)

    db_nb = database_nb[database_nb['Game_Name'] == game]
    db_ct = database_ct[database_ct['Game_Name'] == game]

    fig.add_trace(go.Scatter(x = db_nb['Timeslot'],
                                y = db_nb['Mean'], 
                                name = 'Mean Demand',
                               ))

    fig.add_trace(go.Scatter(x = db_nb['Timeslot'],
                                y = db_nb['Threshold'], 
                                name = 'Threshold',
                               ))

    fig.add_trace(go.Scatter(x = db_nb['Timeslot'],
                                y = (db_nb['Mean']+2*db_nb['Stdev']), 
                                name = 'mu + 2sigma',
                               ))

    fig.add_trace(go.Scatter(x = db_nb['Timeslot'],
                                y = db_nb['Demand'], 
                                name = 'Net Demand',
                             ))
    
    fig.add_trace(go.Scatter(x = db_ct['Peak_Timeslot'],
                             y = (db_ct['Threshold'] + db_ct['Exceeded_MWh']),
                             name = 'Peak Timeslots',
                             mode = 'markers',
                             marker=dict(color='red', size=12),
                            ))
    
    for value in db_ct['Timeslot'].unique():
        fig.add_vline(x=value, line_width=3, line_dash="dash", line_color="green")
                    
    fig.update_yaxes(title_text="Quantity (KWh)")
    fig.update_xaxes(title_text="Timeslot")
    
    fig.update_layout(
        title="Trials1_3_reduced_croc_high_" + game,
        height=750, 
        width=1550, 
        template='ggplot2',
    )
    
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=14)
    
    fig.write_html(storage_path + "/capacity_transaction_Trials1_3_reduced_croc_high_" + game + ".html")

In [ ]:
test_database = dict_of_dataframes.get('VV21').get('Capacity_Transaction_Net_Demand_Info').copy(deep=True)
test_database.Game_Name.unique()

In [ ]:
###############################################
############### Function Call #################
###############################################    
    
capacity_transaction_plotly('VV21', 'finals_2019_07_71_6109')